In [4]:
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import IFCC.data.datapipes as dp
import IFCC.data.tepmanip as tepmanip
import pyreadr as pyr
import importlib
import IFD.units.variational.SS_RNN as SS_RNN
import IFD.support.training as train
from typing import Iterable
import IFD.losses.variational.LossesAndMetrics as loss
import IFCC.models.encoders as enc
import matplotlib.gridspec as gridspec
import pdb
import IFD.detection.varfd as fd
from ipywidgets import interact
import ipywidgets as widgets


2023-12-20 21:00:12.864550: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-20 21:00:12.870269: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-20 21:00:12.966633: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-20 21:00:12.967536: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-20 21:00:14.143551: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [5]:
class test_cell(keras.layers.Layer):

    def __init__(self,num_states,output_dim,layers):
        super().__init__()
        self.num_states=num_states
        self.output_dim=output_dim
        self.t_layers=layers
        self.__weight_init=keras.initializers.RandomNormal()

        self.A=tf.Variable(initial_value=self.__weight_init(shape=(num_states,num_states),dtype=tf.float32),name='A',trainable=True)
        self.B=None
        self.C=tf.Variable(initial_value=self.__weight_init(shape=(num_states,output_dim),dtype=tf.float32),name='A',trainable=True)

    def get_mdl(self):

        return keras.Sequential([keras.layers.Input(shape=[self.num_states,]),
            *self.t_layers,
            keras.layers.Dense(self.num_states)
        ])

    def build(self, input_shape):
        in_shape=input_shape[0]
        state_shape=input_shape[-1]
        #pdb.set_trace()
        self.B=tf.Variable(initial_value=self.__weight_init(shape=[in_shape[-1],self.num_states],dtype=tf.float32),name='B',trainable=True)

        self.mdl_s=self.get_mdl()



    def call(self,inputs):
        u=inputs[0]
        h=inputs[-1]

        h_t=tf.linalg.matmul(h,self.A)+tf.linalg.matmul(u,self.B)
        #pdb.set_trace()
        temp=self.mdl_s(h_t)
        #add losses to regularize the network
        #self.add_loss(tf.linalg.norm(self.A))

        return tf.linalg.matmul(temp,self.C),temp

class test_layer(keras.layers.Layer):

    def __init__(self,cell):
        super().__init__()
        self.cell=cell
        self.num_states=cell.num_states
        self.output_dim=cell.output_dim


    def get_init_state(self,batch_size,dtype):
        return tf.random.normal(shape=[batch_size,self.cell.num_states],dtype=dtype)

    def build(self,input_shape):
        input_shape=(input_shape[-1],)
        state_shape=(self.num_states,)
        self.cell.build([input_shape,state_shape])
    def call(self,x):
        #pdb.set_trace()
        x=tf.transpose(x,perm=[1,0,2]) #[time,batch,features]
        num_time_steps=tf.shape(x)[0]
        batch_size=tf.shape(x)[1]

        state=self.get_init_state(batch_size=batch_size,dtype=x.dtype)
        output_tensor=tf.TensorArray(size=num_time_steps,dtype=x.dtype)

        self.add_loss(tf.linalg.norm(self.cell.A,ord=1))
        self.add_loss(tf.linalg.norm(self.cell.B,ord=1))
        self.add_loss(tf.linalg.norm(self.cell.C,ord=1))

        for i in tf.range(num_time_steps):
            out,state=self.cell([x[i],state])

            output_tensor=output_tensor.write(i,out)

        return output_tensor.stack()







In [6]:
n=52
output_shape=10
inpt_shape=30

dummy_input=tf.random.normal(shape=[64,inpt_shape],dtype='float32')
dummy_state=tf.random.normal(shape=[64,n],dtype='float32')

layers=[keras.layers.Dense(n*2,name='dense_in_mdls')] #,kernel_regularizer=keras.regularizers.L2(0.1)

ts_cell=test_cell(num_states=n,output_dim=output_shape,layers=layers)
ts_layer=test_layer(cell=ts_cell)

dummy_timeseries=tf.random.normal(shape=[10,10,52])

Y=ts_layer(dummy_timeseries)

In [8]:
ts_layer.losses[0]+10

<tf.Tensor: shape=(), dtype=float32, numpy=118.99814>

In [6]:
a=tf.ones(shape=(64,10,52))
b=tf.zeros(shape=(2,64,10,52))
m=keras.metrics.MeanAbsoluteError()
m.update_state(a,b)
print(m.result())

tf.Tensor(1.0, shape=(), dtype=float32)


In [ ]:
print(1)